In [2]:
import numpy as np 
import pandas as pd
import xlsxwriter

print("Fresh Air Calculation Sheet") 
print('\n')

#The leakage coeffcient database based on type of construction

LWC = {'Wall_Tightness':['Tight','Average','Loose','V.Loose'],'Area Ratio':[5.0E-5,1.7E-4,3.5E-4,1.2E-3]}
LFC = {'Floor_Tightness':['Tight','Average','Loose'],'Area Ratio':[6.6E-6,5.2E-5,1.7E-4]}
LW_Coeff = pd.DataFrame(LWC)
LF_Coeff = pd.DataFrame(LFC)
print('Wall Tightness Table')
display(LW_Coeff)
print('Floor & Roof Tightness Table')
display(LF_Coeff)

#Initiation of Fresh Air Calculation.

print('\n')
print('Select the following criteria for fresh air calculation:')
print('Pressurization')
Pressurization = input()
print('Air change per hour')
ACPH = input()
print('Ashrae 62.1')
Ashrae = input()
print('Local Standard')
local_standard = input()

print('Enter the number of Buildings') 
No_of_Buildings = int(input())                                                 #Total No of Buildings in the Project. 
print('The No of Buildings are :', No_of_Buildings) 

Name_of_Buildings = []                                                         #defining names of building. 
for i in range(0,No_of_Buildings): 
    print('the name of building', i) 
    building = input() 
    Name_of_Buildings.append(building) 
    print('The name of buildings are : ', Name_of_Buildings)
    
        
    #Discharge  coeffcients for building or projects.
    if Pressurization == 'yes':
        print('Enter the wall tightness of the building:')
        Ao = str(input())
        print('Enter the discharge coeffcient of walls')
        CW = float(input())
        print('Enter the discharge coeffcient of doors:')
        CD = float(input())
    
    # Creating Door & Window database.
    door_type = []                                       #Creating list of door parameters.
    door_height = []
    door_width = []
    door_area = []
    door_perimeter = []
    Door = {}                                            #d_type, d_l, d_h, d_a,d_p are door type, length, height, area & perimeter respectively.
    print('Enter the types of door in building')
    d_type_num = int(input()) 
    for num in range(0,d_type_num):
        print('Enter the type of door:')
        d_type = input()
        d_type = d_type.upper()
        print('Enter the height and breadth of door')
        d_h = float(input())
        d_w = float(input())
        d_a = d_w*d_h
        
        if d_type == 'SD':
            d_p = 2*(d_h+d_w)
        if d_type == 'DD':
            d_p = 2*(d_h+d_w)+d_h 
            
        door_perimeter.append(d_p)        
        door_type.append(d_type)
        door_height.append(d_h)
        door_width.append(d_w)
        door_area.append(d_a)

            
    print(door_type, door_height, door_width, door_perimeter)   
    Door = {'Type':door_type,'Height':door_height,'Width':door_width,'Area':door_area,'Perimeter':door_perimeter}
    DOOR = pd.DataFrame(Door)
    display(DOOR)
    
    for i in Name_of_Buildings:

        print('Enter the Building orientation;N-S or NE-SW:')
        Building_Orientation = input()
        print('Enter the number of rooms') 
        Rooms_No = int(input())                                        #input number of rooms in building. 
        room_name =[]                                                  #defining the room names.
        pressure = []                                                  #defining room design parameters.units are Pa, degree C.
        temperature = []
        relative_humidity = []
        
        length = []                                                    #defining room parameters.unit is meter (m).
        width = []
        height = []
        false_height = []
        
        Area = []                                                      #defining variables for leakage calculation. unit is m2.
        Volume = []
        exposed_wall = []
        roof = []
        false_flooring = []
        partition_wall = []
        partition_ceiling = []
        
        Wall_Leakage = []                                              #defining leakage variables. unit is m3/hr
        Roof_Leakage = []
        Floor_Leakage = []
        Door_Leakage = []
        Total_Leakage = []
        
        Fresh_Air_Ashrae = []
        Fresh_Air_ACPH = []
        Fresh_Air_LocalStandard = []
        MAX_FRESHAIR = []
        
        Occupancy = []
        
        North_Wall = []                                               #defining exposed wall area. unit is m2
        South_Wall = []
        East_Wall = []
        West_Wall = []
        NorthEast_Wall = []
        SouthEast_Wall = []
        NorthWest_Wall = []
        SouthWest_Wall = []
        
        Electrical_load = []                                          #defining heat dissipation data variables. unit is W.
        Instrument_load = []
        Lighting_load = []
        
        for j in range(0,Rooms_No):
            print('Enter the name of room', j)
            room = input()
            room_name.append(room)
        
        print('\n')
        for k in room_name:

            print('Enter the pressure of the room to be maintained:')
            Pr = int(input())                                               #Pressure of the room in Pa
            print('Enter the design temperature and relative humidity to be maintained', k)
            
            temp = int(input())
            humid = int(input())
            
            pressure.append(Pr)
            temperature.append(temp)
            relative_humidity.append(humid)
            
            print('Enter the Length, Width, Height & False Height of room in "m"', k)
            l = float(input())
            w = float(input())
            h = float(input())
            f_h = float(input())
            
            length.append(l)
            width.append(w)
            height.append(h)
            false_height.append(f_h)
            
            print('Enter the no of Occupants:')
            No_of_Occupants = int(input())
            Occupancy.append(No_of_Occupants)
            print('\n')
            
            A  = l*w                                                  #Area of the room.
            Area.append(A)
            
            if f_h == 0:                                              #calculation of volume of room.
                vol = A*h
            else:
                vol =A*f_h
            Volume.append(vol)
            
            print('\n')
            
            print('Roof & Floor area calculation (m2):')
            print('\n')
            if f_h == 0:
                print('Is the roof of the room partial:')
                if input() == 'yes':
                    print('Enter the coefficients c & d for length and width of room respectively')
                    c = float(input())
                    d = float(input())
                    r = (c*l)*(d*w)
                else:
                    r = A
                    roof.append(r)
            else:
                print('Enter the ceiling partition coeffcients of room')
                x = float(input())
                y = float(input())
                c_p = (x*l)*(y*w)
                partition_ceiling.append(c_p)
                roof.append(c_p)
            print('Is the floor of room raised')
            
            if input() == 'yes':
                if f_h != 0:
                    f = c_p
                else:
                    f = r
            else:
                f = 0
            false_flooring.append(f)
            
            if Pressurization == 'yes':
            
                print('Exposed wall calculation (m2):')              
                print('Enter the coefficients of length and width, a & b respectively:')
                a = float(input())
                b = float(input())
                
                if f_h != 0:
                    exp_wall = ((a*l)+(b*w))*f_h
                else:
                    exp_wall = ((a*l)+(b*w))*h
                    
                exposed_wall.append(exp_wall)
            
                print('Door Clasification and area calculation.')
                print('\n')
                tot_dl = []                                                     #total door leakage of room
                print('Enter the gap of door in "m":')
                d_gap = float(input())
                print('\n')
            
            #Calculation of door leakage.
            
                print('Enter the number for each type of door in room')
            
                d_no = list(map(int,input().split()))
                print(d_no)
            
                for no in d_no:
                    print('Enter the type of door in',k)
                    t = input()
                    t = t.upper()
                    d_p = float(DOOR.query('Type == @t')['Perimeter'])
                    d_lv = round(no*d_gap*d_p*CD*(((2*Pr)/1.2)**0.5)*3600,2)          #leakage for a input door type
                    print('the total leakage door area for', t, 'is :',d_lv)
                    tot_dl.append(d_lv)
                    print('\n')
                    print(tot_dl)
                tot_dlv = sum(tot_dl)
            
            
            #Calculation of wall leakage
            
                area_ratio_for_wall = float(LW_Coeff.query('Wall_Tightness == @Ao')['Area Ratio'])
                w_l = round(3600*CW*exp_wall*area_ratio_for_wall*(((2*Pr)/1.2)**0.5),2)
            
            #Calculation of roof and floor leakage.
                area_ratio_for_R_F = float(LF_Coeff.query('Floor_Tightness == @Ao')['Area Ratio'])
                if f_h == 0:
                    r_l = round(3600*CW*r*area_ratio_for_R_F*(((2*Pr)/1.2)**0.5),2)
                else:
                    r_l = 0
            
                f_l = round(3600*CW*f*area_ratio_for_R_F*(((2*Pr)/1.2)**0.5),2)
                
            #Calcualtion of total leakage due to pressurisation
                FA_leakage = w_l + r_l + f_l + tot_dlv
            
            else: 
                w_l = 0
                r_l = 0
                f_l = 0
                tot_dlv = 0
                FA_leakage = 0
                
            Total_Leakage.append(FA_leakage)
            Door_Leakage.append(tot_dlv)
            Wall_Leakage.append(w_l)
            Roof_Leakage.append(r_l)
            Floor_Leakage.append(f_l)
                
            if ACPH == 'yes':                                                     #ACPH calculation.
                print('Enter the Air change per hour:')
                aph = float(input())
                FA_acph = round((aph*vol)/3.6,2)
                
            else:
                FA_acph = 0
                
            Fresh_Air_ACPH.append(FA_acph)
            
            if Ashrae == 'yes':
                Table_62 = pd.read_excel(r'F:\Parth\HVAC\Ashrae_62.1.xlsx')
                print('Enter the category of room based on Ashrae 62.1 table')
                room_category = str(input())
                Rp = float(Table_62.query('Occupancy_Category == @room_category')['Rp_L/s.person'])
                Ra = float(Table_62.query('Occupancy_Category == @room_category')['Ra_L/s.m2'])
                FA_Ashrae = (Rp*No_of_Occupants)+(Ra*A)
                
            else :
                
                FA_Ashrae = 0
                
            Fresh_Air_Ashrae.append(FA_Ashrae)
                
            if local_standard == 'yes':
                print('Enter "area"/"volume"/value required for fresh air calculation based on local standard.')
                var_1 = input()
                print('Enter the value')
                var_2 = float(input())

                def local_standard():
                    if var_1 == 'area':
                        FA_LS = A*var_2
                    if var_1 == 'volume':
                        FA_LS = V*var_2
                    if var_1 != 'area' and var_1 != 'volume':
                        print('Enter the value')
                        var_0 = float(var_1)
                        FA_LS = var_2*var_0
                    else :
                        FA_LS = 0
                        
                        return FA_LS
                    Fresh_Air_LocalStandard.append(FA_LS)
                
                local_standard()
            
            else :
                
                FA_LS = 0
                
            Fresh_Air_LocalStandard.append(FA_LS)
            
            max_FA = max(FA_LS,FA_Ashrae,FA_leakage,FA_acph)
            MAX_FRESHAIR.append(max_FA)

            #Exposed wall area calculation based on building orientation.
            print('Do you want to consider false ceiling as partition for heat load calculation')
            var_3 = input()
            if var_3 == 'yes':
                exposed_height = f_h
            else:
                exposed_height = h
                
            #Partition wall area calculation
            print('Enter the length of partition wall' )
            var_4 = float(input())
            part_wall = var_4*exposed_height
            partition_wall.append(part_wall)
            
            if Building_Orientation == 'N-S':
                print('Enter the lengths of walls for N,E,S,W facing.')
                var_5 = list(map(int,input().split()))
                N = var_2[0]*exposed_height
                E = var_2[1]*exposed_height
                S = var_2[2]*exposed_height
                W = var_2[3]*exposed_height
                
            else:
                N = E = S = W =0
                
            if Building_Orientation == 'NE-SW':
                print('Enter the lengths of walls for NE,SE,NW,SW facing.')
                var_6 = list(map(int,input().split()))
                NE = var_2[0]*exposed_height
                SE = var_2[1]*exposed_height
                NW = var_2[2]*exposed_height
                SW = var_2[3]*exposed_height
                
            else:
                NE = SE = SW = NW =0
            North_Wall.append(N)
            South_Wall.append(S)
            East_Wall.append(E)
            West_Wall.append(W)
            NorthEast_Wall.append(NE)
            SouthEast_Wall.append(SE)
            NorthWest_Wall.append(NW)
            SouthWest_Wall.append(SW)
            
            print('\n')
            print('Enter the heat dissipation data of Electrical & Instrumentation equipment in Watts:')
            EL = float(input())
            IN = float(input())
            print('\n')
            print('Enter the lighting load in Watts:')
            light_load = float(input())
            Electrical_load.append(EL)
            Instrument_load.append(IN)
            Lighting_load.append(light_load)
                    
print(room_name, pressure, temperature, relative_humidity, Occupancy, length, width, height, false_height, Area, Volume, roof, false_flooring, Wall_Leakage, Roof_Leakage, Door_Leakage, Total_Leakage, Fresh_Air_ACPH, Fresh_Air_Ashrae, Fresh_Air_LocalStandard)
#Creating a Dataframe from of all the parameters and calculated values.
FRESH_AIR = {'Room':room_name, 'Room Pressure': pressure, 'Temperature':temperature, 'RH(%)':relative_humidity,'No of Occupants':Occupancy, 'Length':length, 'Width':width, 'Height':height, 'False Height':false_height, 'Room Area':Area, 'Room Volume':Volume, 'Roof Area':roof, 'False Flooring Area':false_flooring, 'Wall Leakage':Wall_Leakage, 'Roof Leakage':Roof_Leakage, 'Floor Leakage':Floor_Leakage, 'Door Leakage':Door_Leakage, 'Total Leakage':Total_Leakage, 'Fresh Air (ACPH)':Fresh_Air_ACPH, 'Fresh Air Ashrae 62.1':Fresh_Air_Ashrae, 'Fresh_Air_Local':Fresh_Air_LocalStandard}               
FA_Cal = pd.DataFrame(FRESH_AIR)  

#Creating an excel file based on dataframe using xlsxwriter.

writer = pd.ExcelWriter('Fresh_Air_Calculation.xlsx', engine='xlsxwriter')
FA_Cal.to_excel(writer, sheet_name = 'sheet1')
writer.save()


Fresh Air Calculation Sheet


Wall Tightness Table


,Wall_Tightness,Area Ratio
0,Tight,0.00005
1,Average,0.00017
2,Loose,0.00035
3,V.Loose,0.00120


Floor & Roof Tightness Table


,Floor_Tightness,Area Ratio
0,Tight,0.000007
1,Average,0.000052
2,Loose,0.000170




Select the following criteria for fresh air calculation:
Pressurization
no
Air change per hour
yes
Ashrae 62.1
no
Local Standard
no
Enter the number of Buildings
1
The No of Buildings are : 1
the name of building 0
SRR
The name of buildings are :  ['SRR']
Enter the types of door in building
1
Enter the type of door:
dd
Enter the height and breadth of door
2.5
2
['DD'] [2.5] [2.0] [11.5]


,Type,Height,Width,Area,Perimeter
0,DD,2.5,2.0,5.0,11.5


Enter the number of rooms
1
Enter the name of room 0
SRR room


Enter the pressure of the room to be maintained:
20
Enter the design temperature and relative humidity to be maintained SRR room
24
45
Enter the Length, Width, Height & False Height of room in "m" SRR room
15
9
5
3
Enter the no of Occupants:
0




Roof & Floor area calculation (m2):


Enter the ceiling partition coeffcients of room
1
1
Is the floor of room raised
yes
Enter the Air change per hour:
2
['SRR room'] [20] [24] [45] [0] [15.0] [9.0] [5.0] [3.0] [135.0] [405.0] [135.0] [135.0] [0] [0] [0] [0] [225.0] [0] [0]
Enter the Building orientation;N-S or NE-SW
N-S
